In [19]:
from datasets import load_dataset
small_train_dataset = load_dataset('text', data_files='work/traces_dataset/google/*.txt', split='train[:10]')

Resolving data files:   0%|          | 0/395 [00:00<?, ?it/s]

Using custom data configuration default-fc292e433a104d45


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

PermissionError: [Errno 13] Permission denied: '/home/jovyan/work/traces_dataset/google/com_google_common_cache_LocalLoadingCacheTest_data_set.txt.lock'

In [18]:
!ls -allah /home/jovyan/work/traces_dataset/

total 108K
drwxrwsr-x 4 1005 mygroup 4.0K Jan 26 17:27 .
drwxrwsr-x 8 1005 mygroup 4.0K Jan 26 17:25 ..
drwxrwsr-x 2 1005 mygroup  36K Jan 26 17:29 google
drwxrwsr-x 2 1005 mygroup  64K Jan 26 17:32 netty


In [17]:
!touch /home/jovyan/work/traces_dataset/google/com_google_common_cache_LocalLoadingCacheTest_data_set.txt.lock

touch: cannot touch '/home/jovyan/work/traces_dataset/google/com_google_common_cache_LocalLoadingCacheTest_data_set.txt.lock': Permission denied


In [12]:
!id $(whoami)

uid=1000(jovyan) gid=100(users) groups=100(users),1024(mygroup)


In [ ]:
small_train_dataset

In [4]:
df = small_train_dataset.map(lambda x: {'func_name': x["text"].split(' ')[0]})

Loading cached processed dataset at /home/jovyan/.cache/huggingface/datasets/text/default-fd3e663e823696b8/0.0.0/d86c40dad297bdddf277b406c6a59f0250b5318c400bf23d420a31aff88c84c4/cache-6f5d315444157785.arrow


In [1]:
df['func_name']

NameError: name 'df' is not defined

In [12]:
import collections
frequency = collections.Counter(df['func_name']).most_common()


In [13]:
frequency

[('compare', 1120325),
 ('get', 347644),
 ('assertEquals', 199831),
 ('isWellFormed', 196608),
 ('access$100', 180919),
 ('checkPositionIndexes', 139843),
 ('checkNotNull', 133886),
 ('classify', 102795),
 ('start', 99835),
 ('isWellFormedSlowPath', 98304),
 ('result', 96109),
 ('size', 89364),
 ('addTestMethod', 84423),
 ('isPublicTestMethod', 84423),
 ('isTestMethod', 84422),
 ('hasNext', 72503),
 ('lexicographical', 71864),
 ('LexicographicalOrdering', 71864),
 ('doPushState', 66541),
 ('popState', 66299),
 ('getRawType', 63540),
 ('of', 59643),
 ('access$200', 52108),
 ('getName', 49829),
 ('getKey', 49016),
 ('hashCode', 45959),
 ('equals', 41474),
 ('asErasure', 38194),
 ('expiresAfterAccess', 36397),
 ('iterator', 33995),
 ('enterContext', 33607),
 ('close', 33607),
 ('pushDependency', 33365),
 ('popStateAndSetDependency', 33365),
 ('InternalContext', 33302),
 ('pushState', 33176),
 ('checkState', 31938),
 ('next', 31186),
 ('checkPositionIndex', 30560),
 ('evictsBySize', 30025)

In [1]:
from transformers import PreTrainedTokenizerFast
#tokenizer_object=trace_tokenizer
fast_tokenizer = PreTrainedTokenizerFast(tokenizer_file="work/projsplit-train/trace_tokenizer.json",
                                         return_special_tokens_mask=True, mask_token='[MASK]', return_token_type_ids=False)
fast_tokenizer.add_special_tokens({'pad_token': '[PAD]', 'mask_token': '[MASK]'})

0

In [17]:
from transformers import BertConfig, BertModel, BertForMaskedLM
bertmodel = BertModel.from_pretrained('work/projsplit-train/test_trainer_bert_pre')
bertmodel.tokenizer = fast_tokenizer
bertmodel.config.output_hidden_states = True

Some weights of the model checkpoint at work/projsplit-train/test_trainer_bert_pre were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at work/projsplit-train/test_trainer_bert_pre and are newly initialized: ['bert.poole

In [1]:
small_train_dataset["text"][:10]

NameError: name 'small_train_dataset' is not defined

In [ ]:
bertmodel(**fast_tokenizer(small_train_dataset["text"],
                                   max_length = 512, truncation=True, padding=True, return_tensors="pt"
                                  )).hidden_states[-1].shape